# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails 
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice.aci import AciWebservice
from azureml.core.webservice import Webservice
import os
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
if not os.path.exists('results'):
    os.makedirs('results')

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'predict-breast-cancer-automl'

# Check workspace details
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

experiment=Experiment(ws, experiment_name)

Workspace name: quick-starts-ws-142844
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-142844


In [4]:
# Check for existing cluster. Otherwise, create new cluster
try:
    cluster = ComputeTarget(workspace=ws, name="notebook142844")
    print("Cluster exists")
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V12', max_nodes=4)
    cluster = ComputeTarget.create(ws, "project-cluster", config)

cluster.wait_for_completion()

Cluster exists


In [5]:
train_data = Dataset.get_by_name(ws, name="breast_cancer_data")
train_data.take(5).to_pandas_dataframe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cluster,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="diagnosis",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             blocked_models=['XGBoostClassifier'],
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on notebook142844 with default configuration
Running on remote compute: notebook142844


Experiment,Id,Type,Status,Details Page,Docs Page
predict-breast-cancer-automl,AutoML_1678d470-d521-451d-903f-e814ce11151b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
predict-breast-cancer-automl,AutoML_1678d470-d521-451d-903f-e814ce11151b,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_1678d470-d521-451d-903f-e814ce11151b',
 'target': 'notebook142844',
 'status': 'Completed',
 'startTimeUtc': '2021-04-16T12:01:01.000141Z',
 'endTimeUtc': '2021-04-16T12:52:57.809338Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook142844',
  'AMLSettingsJsonString': '{"path":null,"name":"predict-breast-cancer-automl","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-142844","workspace_name":"quick-starts-ws-142844","region":"southcentralus","compute_target":"notebook142844","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_automl_run, best_automl_model = remote_run.get_output()
best_automl_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('5',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f5711a51da0>),
                                                           ('sgdclassifierwrapper',
                                                            SGDClassifierWrapper(alpha=1.6327367346938775,
                                                                                 class_weight='balanced',
                                                                                 eta0=1e-05,
                                                                                 fit_intercept=True,
                                                                                 l1_rati...
     

In [14]:
print(best_automl_run)
model_name = best_automl_run.properties['model_name']
print("Best model: ", model_name)
best_automl_model.get_params()

Run(Experiment: predict-breast-cancer-automl,
Id: AutoML_1678d470-d521-451d-903f-e814ce11151b_25,
Type: azureml.scriptrun,
Status: Completed)
Best model:  AutoML1678d470d25


{'memory': None,
 'steps': [('datatransformer',
   DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                   feature_sweeping_config=None, feature_sweeping_timeout=None,
                   featurization_config=None, force_text_dnn=None,
                   is_cross_validation=None, is_onnx_compatible=None, logger=None,
                   observer=None, task=None, working_dir=None)),
  ('prefittedsoftvotingclassifier',
   PreFittedSoftVotingClassifier(classification_labels=None,
                                 estimators=[('5',
                                              Pipeline(memory=None,
                                                       steps=[('standardscalerwrapper',
                                                               <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f5711a51da0>),
                                                              ('sgdclassifierwrapper',
                                         

In [15]:
#TODO: Save the best model
joblib.dump(best_automl_model, filename="results/automl_breast_cancer_model.joblib")

['results/automl_breast_cancer_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service